In [1]:
import numpy as np
from numpy import linalg
from sympy import *

In [2]:
# Rossenbrock function
# f1(x) = 10*(x2 - x1^2)
# f2(x) = 1 - x1
# x0 = (-1.2, 1)
# f = 0 at (1, 1)
# n = 2, m = 2 13

In [3]:
# Freudenstein and Roth function
# f1(x) = -13 + x1 + ((5 - x2)*x2 - 2)*x2
# f2(x) = -29 + x1 + ((x2 + 1)*x2 - 14)*x2
# x0 = (0.5, -2)
# f = 0 at (5, 4)

In [4]:
# Powell badly scaled function
# f1(x) = 10**4*x1*x2 - 1
# f2(x) = exp(-x1) + exp(-x2) - 1.0001
# x0 = (0, 1)
# f = 0 at (1.098...10**-5, 9.106...)

In [5]:
# Powell singular function
# f1(x) = x1 + 10*x2
# f2(x) = 5^(1/2)*(x3 - x4)
# f3(x) = (x2 - 2*x3)^2
# f4(x) = 10^(1/2)*(x1 - x4)^2
# x0 = [3, -1, 0, 1]
# f = 0

In [6]:
x1 = symbols('x1')
x2 = symbols('x2')
x3 = symbols('x3')
x4 = symbols('x4')

# f1 = simplify("10*(x2 - x1^2)")
# f2 = sympify("1 - x1")
# x0 = np.array([-3, 2], dtype = 'float')

# f1 = simplify("(10**4)*x1*x2 - 1")
# f2 = simplify("exp(-x1) + exp(-x2) - 1.0001")
# x0 = np.array([0, 1], dtype = 'float')

# f1 = simplify("-13 + x1 + ((5 - x2)*x2 - 2)*x2")
# f2 = simplify("-29 + x1 + ((x2 + 1)*x2 - 14)*x2")
# x0 = np.array([0.5, -4], dtype = 'float')

f1 = simplify('x1 + 10*x2')
f2 = simplify('5^(1/2)*(x3 - x4)')
f3 = simplify('(x2 - 2*x3)^2')
f4 = simplify('10^(1/2)*(x1 - x4)^2')
x0 = np.array([3, -1, 0, 1], dtype = 'float')

# x = np.array([x1, x2])
# F = np.array([f1, f2])

x = np.array([x1, x2, x3, x4])
F = np.array([f1, f2, f3, f4])

In [7]:
# n = int(input("Enter n = "))
# x = np.array([symbols('x{0}'.format(i + 1)) for i in range(n)])

# F = np.array([simplify(input("f{0} = ".format(i + 1))) for i in range(n)])
# x0 = np.array([float(input("{0} Element of x0".format(i + 1))) for i in range(n)], dtype = 'float')

eps = 10 ** -7

In [8]:
F_0 = lambda x0: np.array([F[i].subs(dict(zip(x, x0))) for i in range(len(F))], dtype = 'float')

In [9]:
def F_der_0(x0):
    F_der = np.array([[diff(F[i], x[j]) for j in range(len(x))] for i in range(len(F))])
    return np.array([[F_der[i][j].subs(dict(zip(x, x0))) for j in range(len(F_der[i]))] for i in range(len(F_der))], dtype = 'float')

In [10]:
# test function for F(x, y)
# A = np.array([simplify("x1**2 + 7*x1 + 2*x1*x2 - 3"), simplify("x1 + x2**3")])
# A = np.array([simplify("-13 + x1 + ((5 - x2)*x2 - 2)*x2"), simplify("-29 + x1 + ((x2 + 1)*x2 - 14)*x2")])
# A[0].subs({x1: 1.13975640, x2: 1.05684169})

In [11]:
# Поділена різниця першого порядку
def F_1(x1, x2):
    assert(len(x1) == len(x2))
    arr = []
    for i in range(len(F)):
        subarr = []
        for j in range(len(x1)):
            params1 = [*x1[0:j+1], *x2[j+1:len(x2)+1]]
            params2 = [*x1[0:j], *x2[j:len(x2)+1]]
            # print("x1[j] = {0}, x2[j] = {1}, x1[j] - x2[j] = {2}".format(x1[j], x2[j], x1[j] - x2[j]))
            # print((F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))))
            result = (F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))) / (x1[j] - x2[j])
            subarr.append(result)
        
        arr.append(subarr)
    
    return np.array(arr, dtype = 'float')


In [12]:
def NewthonMethod(x0, x, F):
    I = 0

    x1 = x0 - np.dot(np.linalg.inv(F_der_0(x0)), F_0(x0))
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I +=1
    
    while True:
        x0 = x1
        x1 = x0 - np.dot(np.linalg.inv(F_der_0(x0)), F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [13]:
def PotraMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    I = 0

    matrix = (F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1)
    x1 = x0 - np.dot(np.linalg.inv(matrix), F_0(x0))
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I += 1

    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1

        matrix = (F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1)
        x1 = x0 - np.dot(np.linalg.inv(matrix), F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0


In [14]:
# Inverse operator methods
def SequentialNewthonInverseOperatorMethod(x0, x, F):
    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv(F_der_0(x0))

    x1 = x0 - np.dot(A_0, F_0(x0))
    right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))

    number_of_iterations += 1
    
    while True:
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
        A_1 = np.dot(A_0, right_matrix)
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1

    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0
    

In [15]:
def ParallelNewthonInverseOperatorMethod(x0, x, F):
    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv(F_der_0(x0))

    x1 = x0 - np.dot(A_0, F_0(x0))
    right_matrix = 2 * I - np.dot(F_der_0(x0), A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))

    number_of_iterations += 1
    
    while True:
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        right_matrix = 2 * I - np.dot(F_der_0(x0), A_0)
        A_1 = np.dot(A_0, right_matrix)
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1

    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0

In [16]:
def SequentialPotraInverseOperatorMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv((F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1))

    x1 = x0 - np.dot(A_0, F_0(x0))
    matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
    # right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
    right_matrix = 2 * I - np.dot(matrix, A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
    number_of_iterations +=1
    
    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
        # right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
        right_matrix = 2 * I - np.dot(matrix, A_0)
        A_1 = np.dot(A_0, right_matrix)
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1
    
    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0


In [17]:
def ParallelPotraInverseOperatorMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv((F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1))

    x1 = x0 - np.dot(A_0, F_0(x0))
    # matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
    matrix = (F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1)
    # right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
    right_matrix = 2 * I - np.dot(matrix, A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
    number_of_iterations +=1
    
    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        # matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
        matrix = (F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1)
        # right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
        right_matrix = 2 * I - np.dot(matrix, A_0)
        A_1 = np.dot(A_0, right_matrix)
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1
    
    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0

In [18]:
print("Метод Ньютона з послідовною апроксимацією оберненого оператора\n\n")
a = SequentialNewthonInverseOperatorMethod(x0, x, F)
print(a)
# print("Норма відхилу: {0}".format(max(np.abs(a - x0))))

Метод Ньютона з послідовною апроксимацією оберненого оператора


1 Ітерація: Значення F = [-2.22044605e-16  1.86190061e-16  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [0.         0.         0.09765625 1.23526471] у точці x = [ 0.74404762 -0.07440476  0.11904762  0.11904762]
3 Ітерація: Значення F = [0.         0.         0.04013928 0.50772621] у точці x = [ 0.47701881 -0.04770188  0.07632301  0.07632301]
4 Ітерація: Значення F = [5.55111512e-17 0.00000000e+00 1.67355214e-02 2.11689462e-01] у точці x = [ 0.30801391 -0.03080139  0.04928223  0.04928223]
5 Ітерація: Значення F = [0.         0.         0.00700849 0.08865119] у точці x = [ 0.19932556 -0.01993256  0.03189209  0.03189209]
6 Ітерація: Значення F = [-2.77555756e-17  0.00000000e+00  2.93912541e-03  3.71773225e-02] у точці x = [ 0.12908024 -0.01290802  0.02065284  0.02065284]
7 Ітерація: Значення F = [-1.38777878e-17  0.00000000e+00  1.23312296e-03  1.5597

In [19]:
print("Метод Ньютона з паралельною апроксимацією оберненого оператора\n\n")
a = ParallelNewthonInverseOperatorMethod(x0, x, F)
print(a)
# print("Норма відхилу: {0}".format(a - x0))

Метод Ньютона з паралельною апроксимацією оберненого оператора


1 Ітерація: Значення F = [-2.22044605e-16  1.86190061e-16  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [0.         0.         0.140625   1.77878118] у точці x = [ 0.89285714 -0.08928571  0.14285714  0.14285714]
3 Ітерація: Значення F = [0.         0.         0.07264709 0.91892114] у точці x = [ 0.64174107 -0.06417411  0.10267857  0.10267857]
4 Ітерація: Значення F = [0.         0.         0.03656073 0.46246071] у точці x = [ 0.45525857 -0.04552586  0.07284137  0.07284137]
5 Ітерація: Значення F = [0.         0.         0.01830143 0.23149687] у точці x = [ 0.32210189 -0.03221019  0.0515363   0.0515363 ]
6 Ітерація: Значення F = [2.77555756e-17 0.00000000e+00 9.15253599e-03 1.15771440e-01] у точці x = [ 0.22778306 -0.02277831  0.03644529  0.03644529]
7 Ітерація: Значення F = [-2.77555756e-17  0.00000000e+00  4.57642419e-03  5.78876959e-02] у точці x 

In [20]:
print("Метод Потра з послідовною апроксимацією оберненого оператора\n\n")
a = SequentialPotraInverseOperatorMethod(x0, x, F)
print(a)
# print("Норма відхилу: {0}".format(max(np.abs(a - x0))))

Метод Потра з послідовною апроксимацією оберненого оператора


1 Ітерація: Значення F = [-8.14037726e-12  4.74164023e-14  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [2.64344102e-13 5.83395526e-15 9.76562500e-02 1.23526471e+00] у точці x = [ 0.74404762 -0.07440476  0.11904762  0.11904762]
3 Ітерація: Значення F = [5.47895063e-14 1.17920372e-15 4.01392812e-02 5.07726209e-01] у точці x = [ 0.47701881 -0.04770188  0.07632301  0.07632301]
4 Ітерація: Значення F = [1.21569421e-14 2.79285092e-16 1.67355214e-02 2.11689462e-01] у точці x = [ 0.30801391 -0.03080139  0.04928223  0.04928223]
5 Ітерація: Значення F = [2.74780199e-15 6.20633538e-17 7.00849198e-03 8.86511904e-02] у точці x = [ 0.19932556 -0.01993256  0.03189209  0.03189209]
6 Ітерація: Значення F = [6.10622664e-16 1.55158385e-17 2.93912541e-03 3.71773225e-02] у точці x = [ 0.12908024 -0.01290802  0.02065284  0.02065284]
7 Ітерація: Значення F = [1.66533454e-1

In [21]:
print("Метод Потра з паралельною апроксимацією оберненого оператора\n\n")
a = ParallelPotraInverseOperatorMethod(x0, x, F)
print(a)
# print("Норма відхилу: {0}".format(max(np.abs(a - x0))))

Метод Потра з паралельною апроксимацією оберненого оператора


1 Ітерація: Значення F = [-8.14037726e-12  4.74164023e-14  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [-1.85029769e-12 -1.16679105e-14  1.40625000e-01  1.77878118e+00] у точці x = [ 0.89285714 -0.08928571  0.14285714  0.14285714]
3 Ітерація: Значення F = [1.48658863e-13 3.25832608e-15 7.26470947e-02 9.18921139e-01] у точці x = [ 0.64174107 -0.06417411  0.10267857  0.10267857]
4 Ітерація: Значення F = [3.35842465e-14 7.13728569e-16 3.65607291e-02 4.62460707e-01] у точці x = [ 0.45525857 -0.04552586  0.07284137  0.07284137]
5 Ітерація: Значення F = [7.04991621e-15 1.55158385e-16 1.83014344e-02 2.31496868e-01] у точці x = [ 0.32210189 -0.03221019  0.0515363   0.0515363 ]
6 Ітерація: Значення F = [1.44328993e-15 4.65475154e-17 9.15253599e-03 1.15771440e-01] у точці x = [ 0.22778306 -0.02277831  0.03644529  0.03644529]
7 Ітерація: Значення F = [3.0531133

In [22]:
# print("Метод Ньютона\n\n")
# print(NewthonMethod(x0, x, F))

In [23]:
# print("Метод Потра\n\n")
# print(PotraMethod(x0, x, F))